In [ ]:
# 패키지 설치
!pip install yfinance --quiet

In [ ]:
# 패키지 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from scipy.optimize import minimize
sns.set()

## 데이터셋 준비

총 16개 ETF에 대한 가격 데이터를 다운받아오도록 하겠습니다. 여기서 사용하게 될 데이터는 전부 월간 단위 데이터입니다. 

In [ ]:
# ETF 데이터 다운로드
tickers = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']
etf = yf.Tickers(tickers)
data = etf.history(start='2010-01-01', actions=False)
data.drop(['Open', 'High', 'Low', 'Volume'], inplace=True, axis=1)
data = data.droplevel(0, axis=1).resample('M').last()

In [ ]:
# 수익률 계산
rets = data.pct_change().fillna(0)

In [ ]:
# 색깔 팔레트
pal = sns.color_palette('Spectral', len(tickers))

### 리스크 패리티 포트폴리오 최적화 알고리즘

In [ ]:
# 공분산행렬
cov = rets.cov() * 12

# 초기값 설정
noa = rets.shape[1]
init_guess = np.repeat(1/noa, noa)


# 상하한선
bounds = ((0.0, 1.0), ) * noa

# 제약조건
weights_sum_to_1 = {'type': 'eq',
                    'fun': lambda weights: np.sum(weights) - 1}

# 목표 위험기여도 : 동등 위험 기여
target_risk = np.repeat(1/noa, noa)

# 목적 함수 : 목표 위험 기여도와의 오차 최소화
def msd_risk(weights, target_risk, cov):

    port_var = weights.T @ cov @ weights
    marginal_contribs = cov @ weights

    risk_contribs = np.multiply(marginal_contribs, weights.T) / port_var

    return ((risk_contribs - target_risk) ** 2).sum()

# 가중치 산출
res = minimize(msd_risk,
               init_guess,
               args=(target_risk, cov),
               method='SLSQP',
               constraints=(weights_sum_to_1,),
               bounds=bounds)

weights = res.x


In [ ]:
# RP 가중치 데이터프레임 생성
weights_df = pd.Series(np.round(weights, 4), index=tickers)
weights_df = weights_df[weights_df > 0.0]

# 파이차트 시각화
plt.figure(figsize=(7,7))
wedgeprops = {'width':0.3, 'edgecolor':'w', 'linewidth':2}
plt.pie(weights_df, labels=weights_df.index, autopct='%.1f%%', wedgeprops=wedgeprops, colors=pal)
plt.show()

In [ ]:
# 리스크 패리티 포트폴리오 가중치 계산 함수
def get_rp_weights(cov):

    # 자산 개수
    noa = cov.shape[0]

    # 초기 가중치값
    init_guess = np.repeat(1/noa, noa)

    # 제약조건 및 상하한값
    bounds=((0.0, 1.0), ) * noa
    weights_sum_to_1 = {'type':'eq',
                        'fun': lambda weights: np.sum(weights) - 1}

    # 자산별 목표위험 예산 (1/N)
    target_risk = np.repeat(1/noa, noa)

    # 목적함수: 타겟 위험 예산과 자산별 위험 기여도 간의 괴리
    def msd_risk(weights, target_risk, cov):
        port_var = weights.T @ cov @ weights
        marginal_contribs = cov @ weights

        risk_contribs = np.multiply(marginal_contribs, weights.T) / port_var

        return ((risk_contribs - target_risk) ** 2).sum()

    # 최적화 수행
    res = minimize(msd_risk,
                   init_guess,
                   args=(target_risk, cov),
                   method='SLSQP',
                   constraints=(weights_sum_to_1),
                   bounds=bounds)
    
    return res.x

In [ ]:
# 빈 데이터프레임 생성
rp_w_df = pd.DataFrame().reindex_like(rets)

# 공분산행렬의 3차원 배열 생성
cov = rets.rolling(12).cov().fillna(0) * 12
cov = cov.values.reshape(int(cov.shape[0] / cov.shape[1]), cov.shape[1], cov.shape[1])

# RP 가중치 산출
for i in range(12, len(rp_w_df)):
    rp_w_df.iloc[i] = get_rp_weights(cov[i-1])

In [ ]:
# 시계열 플랏 시각화
plt.figure(figsize=(20, 5))
plt.stackplot(rp_w_df.index, rp_w_df.T, labels=rp_w_df.columns, colors=pal)
plt.legend(loc='upper left')
plt.title('RP Weights')
plt.xlabel('Date')
plt.ylabel('Returns')

In [ ]:
# RP 포트폴리오 수익률
port_rets = rp_w_df.shift() * rets
port_cum_rets = (1 + port_rets.sum(axis=1)).cumprod() - 1

# 백테스팅 시각화
plt.plot(port_cum_rets.iloc[12:])
plt.title('RP Backtest')
plt.xlabel('Date')
plt.ylabel('Returns')

Copyright 2022. 퀀트대디. All rights reserved.